In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from today

last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0] 
date_latest = dt.datetime.strptime(last_date , '%Y-%m-%d')
date_year_ago =  date_latest - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores

precipitation = session.query(Measurement.date, 
                              func.avg(Measurement.prcp)).filter(Measurement.date <= date_latest, 
                                                                                   Measurement.date > date_year_ago).group_by(Measurement.date).order_by(Measurement.date).all()   
# Save the query results as a Pandas DataFrame and set the index to the date column

df = pd.DataFrame(precipitation, columns=['Date', 'Total Precipitation'])
df.set_index('Date', inplace=True)

# Sort the dataframe by date
df.head()
# Use Pandas Plotting with Matplotlib to plot the data

# Rotate the xticks for the dates


,Total Precipitation
Date,
2016-08-24,1.555000
2016-08-25,0.077143
2016-08-26,0.016667
2016-08-27,0.064000
2016-08-28,0.516667


In [ ]:
df.plot.bar(figsize = (20,10), linewidth = 10)
plt.ylabel('date')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.tight_layout()
plt.grid(False)
plt.show()

![precipitation](Images/precipitation.png)

In [11]:
df.describe()

,Total Precipitation
count,365.000000
mean,0.169987
std,0.295722
min,0.000000
25%,0.008571
50%,0.070000
75%,0.191667
max,2.380000


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


In [12]:
# How many stations are available in this dataset?
stations_count = session.query(Station.station).group_by(Station.station).count()

In [13]:
active_stations = session.query(Measurement.station, 
                                func.count(Measurement.date)).group_by(Measurement.station).order_by(func.count(Measurement.date)).all()
active_stations

[('USC00518838', 511),
 ('USC00517948', 1372),
 ('USC00511918', 1979),
 ('USC00514830', 2202),
 ('USC00516128', 2612),
 ('USC00519523', 2669),
 ('USC00513117', 2709),
 ('USC00519397', 2724),
 ('USC00519281', 2772)]

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.


In [14]:
most_active_station = active_stations[-1][0]

temp_sel = [func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)]

most_active_station_details = session.query(*temp_sel).filter(Measurement.station == most_active_station).all()
most_active_station_details

[(54.0, 85.0, 71.66378066378067)]

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?


In [27]:
temp_date = session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == most_active_station, 
                                                                     Measurement.date <= date_latest, 
                                                                     Measurement.date > date_year_ago).all()
temp_df = pd.DataFrame(temp_date, columns=['Date', 'Temperature'])
temp_df.set_index('Date', inplace=True)
temp_df.head()

,Temperature
Date,
2016-08-24,77.0
2016-08-25,80.0
2016-08-26,80.0
2016-08-27,75.0
2016-08-28,73.0


In [30]:
year_month = []
for key, value in temp_df.iterrows():
    date = dt.datetime.strptime(key, "%Y-%m-%d")
    year_month.append(str(date.year) + "-" + str(date.month))

        
temp_df["year"] = year_month
temp_df.reset_index(inplace=True)

In [36]:
temp_df.drop('Date', axis=1, inplace=True)

In [46]:
grouped = temp_df.set_index('year').groupby('year')
grouped.head()

,Temperature
year,
2016-8,77.0
2016-8,80.0
2016-8,80.0
2016-8,75.0
2016-8,73.0
2016-9,80.0
2016-9,80.0
2016-9,78.0
2016-9,78.0


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
